In [1]:
using Ferrite, SparseArrays
using FerriteGmsh

In [2]:
const _circuit_types = [:current, :voltage]

2-element Vector{Symbol}:
 :current
 :voltage

In [3]:
struct CircuitCoupling{T} 
    ctype::Symbol
    domain::String
    value::T
end

In [7]:
struct CCouplingHandler{DH<:Ferrite.AbstractDofHandler,T}
    ctype::DataType
    coupling::Vector{CircuitCoupling{T}}
    dh::DH
end

function CCouplingHandler(dh::Ferrite.AbstractDofHandler, val_type::DataType)
    @assert Ferrite.isclosed(dh)
    CCouplingHandler(val_type, CircuitCoupling{val_type}[], dh)
end

function Base.show(io::IO, ::MIME"text/plain", cch::CCouplingHandler)
    println(io, "CCouplingHandler:")
    print(io, "  Circuit constraints:")
    for c in cch.coupling
        print(io, "\n    ", "Domain: ", c.domain, ", ")
        if(c.ctype == :current)
            print(io, "Current: ", c.value, " A")
        elseif(c.ctype == :voltage)
            print(io, "Voltage: ", c.value, " V")
        end
    end
end

In [45]:
function add_current_coupling!(cc::CCouplingHandler, dom_name, current)
    push!(cc.coupling, CircuitCoupling{cc.ctype}(:current, dom_name, current))
end

function apply_circuit(K::SparseMatrixCSC, f::Vector, cc::CCouplingHandler)
    N  = size(K)
    nc = length(cc.coupling)
    K  = hcat(K, spzeros(N[1], nc))
    K  = vcat(K, spzeros(nc, N[2] + nc))
    f  = vcat(f, spzeros(nc))
    
    for circ in cc.coupling
        
    end
    
    return K, f
end

LoadError: syntax: invalid syntax "catch {error("No domain called ", dom_name, "\n")}"

In [39]:
# Load the mesh using FerriteGmsh
grid = saved_file_to_grid("geo/cable_geo.msh");

dh = DofHandler(grid)

push!(dh, :A, 1)
close!(dh)

dim   = 2
order = 1;
ip_fe  = Lagrange{dim, RefTetrahedron, order}()
ip_geo = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(2 * order)
cellvalues = CellScalarValues(qr, ip_fe, ip_geo);

Info    : Reading 'geo/cable_geo.msh'...
Info    : 25 entities
Info    : 16928 nodes
Info    : 33626 elements
Info    : Done reading 'geo/cable_geo.msh'


In [40]:
# Define the boundary conditions using a constraint handler
ch = ConstraintHandler(dh);

dbc1 = Dirichlet(
    :A,
    getfaceset(grid, "jacket"),
    (x,t) -> 0;
);


# Add boundary conditions to the constraint handler and update it
add!(ch, dbc1)

close!(ch)
update!(ch, 0.0); # Since the BCs do not depend on time, update them once at t = 0.0

In [41]:
cc = CCouplingHandler(dh, Complex{Float64});
add_current_coupling!(cc, "Conductor", 1000.0 * exp(1im * 0))

cc

CCouplingHandler:
  Circuit constraints:
    Domain: Conductor, Current: 1000.0 + 0.0im A

In [42]:
# Element assembly: computes the elementary matrix contributions Ke and fe
function assemble_element!(Ke::Matrix, Me::Matrix, fe::Vector, cellvalues::CellScalarValues, Jz, nu, sigma, omega)
    n_basefuncs = getnbasefunctions(cellvalues)
    
    # Reset to 0
    fill!(Ke, 0)
    fill!(Me, 0)
    fill!(fe, 0)
    
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            v  = shape_value(cellvalues, q_point, i)
            ∇v = shape_gradient(cellvalues, q_point, i)
            
            # Add contribution to fe
            fe[i] += Jz * v * dΩ
            
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                u  = shape_value(cellvalues, q_point, j)
                ∇u = shape_gradient(cellvalues, q_point, j)
                
                # Add contribution to Ke & Me
                Ke[i, j] += nu * (∇v ⋅ ∇u) * dΩ
                Me[i, j] += 1im * sigma * omega * (v ⋅ u) * dΩ
            end
        end
    end
    
    return Ke, fe
end

# Global assembly: computes global matrix K and f using element contributions
#  This is quite fast because of the use of a pre-allocated sparse matrix pattern.
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(Complex{Float64}, n_basefuncs, n_basefuncs)
    Me = zeros(Complex{Float64}, n_basefuncs, n_basefuncs)
    fe = zeros(Complex{Float64}, n_basefuncs)
    
    # Allocate global force vector f
    f = zeros(Complex{Float64}, ndofs(dh))
    
    # Create an assembler
    assembler = start_assemble(K, f)
    
    # Loop over all cells
    for cell in CellIterator(dh)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        cid = cellid(cell);
        
        # Compute element contribution
        assemble_element!(Ke, Me, fe, cellvalues, Jz[cid], nu[cid], sigma[cid], omega)
        Ae = Ke + Me;
        
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ae, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

In [43]:
# Define element properties
I = 1000;

Ncells = length(dh.grid.cells);
Jz = zeros(Float64, Ncells);

sigma = zeros(Ncells);
sigma[collect(getcellset(grid, "Conductor"))] .= 36.9e6;
sigma[collect(getcellset(grid, "Sheath"))] .= 59.6e6;

mu0 = 4e-7 * pi;
mur = ones(Ncells);
nu  = 1 ./ (mu0 * mur);

omega = 2*pi * 50;

# Create sparsity pattern from mesh data and assemble the linear system
K = create_sparsity_pattern(dh);

K = convert(SparseMatrixCSC{Complex{Float64}, Int64}, K)
K, f = assemble_global(cellvalues, K, dh);

# Apply the boundary conditions
apply!(K, f, ch)

# Apply the circuit coupling
K, f = apply_circuit(K, f, cc)

# Solve the linear system
u = K \ f;

LoadError: LinearAlgebra.SingularException(0)

In [44]:
K

16929×16929 SparseMatrixCSC{ComplexF64, Int64} with 117958 stored entries:
⣿⣿⣿⣿⡀⠀⠈⠀⠂⡀⢀⣈⣘⠀⠀⠀⠁⠂⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣿⣿⣿⣿⣋⣐⣛⣐⣚⣀⣂⣚⣘⣁⣁⣀⣂⣂⣈⣁⣀⣈⣀⣀⣀⣀⣀⣀⣀⣀⣁⡀⢀⣀⡀⠀⠀⠀⠀⠀
⠀⠈⢋⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠐⢾⠀⠀⠀⠀⠀⠀
⠂⠀⢛⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⠰⠝⠀⠀⠀⠀⠀⠀
⠈⠠⠚⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⢀⢈⠀⠀⠀⠀⠀⠀
⡀⢰⣨⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀
⠒⠘⠖⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡗⢒⣒⡀⠀⠀⠀⠀⠀
⠀⠀⠁⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠐⠀⠂⠀⠀⠀⠀⠀
⠡⠀⠨⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣮⣺⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀
⠁⠀⠆⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠬⠀⠁⠀⠀⠀⠀⠀
⠀⠀⡀⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠒⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⣦⣤⡄⡆⠀⠀⠀⠀
⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣪⣻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡯⠭⢭⠅⠀⠀⠀⠀⠀
⠀⠀⠁⠸⠿⠿⡿⠿⠿⠿⠿⠿⢿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⣿⡿⡿⡿⣯⣭⣽⣧⣆⡀⣀⣀⡀
⠀⠀⠀⢰⣰⣄⣔⠆⡀⢐⠀⠀⢸⢰⠐⠀⠀⠀⠂⠃⠀⠀⠀⠀⠘⠀⠈⣿⡇⣇⣇⣿⣿⣿⣿⣿⠚⠶⠾⠇
⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠈⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠠⠭⠁⠁⠩⢿⣿⣿⣿⣿⣟⣟⣻⡃
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢨⢺⡄⣿⢽⣿⣿⣿⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⠾⠇⠿⠺⠿⠿⠿⠇